# Extrapolación de Richardson

Es un método simple para mejorar la precisión de ciertos procedimientos numéricos, incluyendo aproximaciones de diferencias finitas y ahora, como veremos, en las cuadraturas de Newton-Cotes.

La situación genérica es la siguiente: supongamos que tenemos una manera aproximada de calcular una cantidad cualquiera que llamamos $G$. Más aún, asumimos que el resultado depende del parámetro $h$. Denotamos la aproximación como $g(h)$, entonces tenemos $G=g(h) + E(h)$, donde $E(h)$ es el error de la aproximación.

La **extrapolación de Richardson** puede remover el error en esta aproximación, siempre y cuando sea de la forma $E(h)=ch^p$, donde $c$ y $p$ son constantes.

Comenzamos calculando $g(h)$ para un cierto valor de $h=h_1$ y en ese caso, tenemos
\begin{equation}
G= g(h_1) + ch_1^p.
\end{equation}

Despues repetimos el cálculo para $h=h_2$, asi que
\begin{equation}
G= g(h_2) + ch_2^p.
\end{equation}

Eliminamos $c$ de estas ecuaciones y resolvemos para $G$ para tener la **fórmula de extrapolación de Richardson**:

\begin{equation}
G = \frac{(h_1/h_2)^pg(h_2) - g(h_1)}{(h_1/h_2)^p - 1}.
\end{equation}

Es comun escoger $h_2=h_1/2$, en cuyo caso, la expresión anterior queda

\begin{equation}
G = \frac{2^p g(h_1/2) - g(h_1)}{2^p - 1}. \label{eq:richard}
\end{equation}

# Integración de Romberg

La integración de Romberg combina la regla trapezoidal con la extrapolación de Richardson obtenida en la ecuación para $G$. 

Para desarrollar el algoritmo combinado, utilizamos la notación $R_{i,1} = I_i$, donde $I_i$ representa el valor aproximado de la integral definida dada por la regla del trapezoide usando $2^{i-1}$ paneles. 

Recordemos que el error en esta aproximación es $E=c_1h^2 + c_2h^4 + \ldots$, donde

$$
h = \frac{b-a}{2^{i-1}}
$$
es la anchura de un panel.

La integración de Romberg comienza con el cálculo de $R_{1,1} = I_1$ (un panel) y de $R_{2,1} = I_2$ (dos paneles) de la regla trapezoidal. El error esta dominado por el término $c_1 h^2$, y ese es el que eliminaremos con la extrapolación de Richardson a partir de la ecuación para $G$ aplicado al siguiente valor $R_{2,2}$ que nos interesa

\begin{equation}
R_{2,2} = \frac{2^2 R_{2,1} - R_{1,1}}{2^{2-1}}. \label{eq:romberg1}
\end{equation}

Es conveniente guardar los resultados en un arreglo de la forma

\begin{equation}
\begin{bmatrix}
R_{1,1} & \\
R_{2,1} & 	R_{2,2}
\end{bmatrix} \label{eq:romberg2}
\end{equation}

El siguiente paso es calcular $R_{3,1} = I_3$ (cuatro paneles) y repetir la extrapolación con $R_{2,1}$ y $R_{3,1}$, para guardar el resultado $R_{3,2}$ como 

\begin{equation}
R_{3,2} = \frac{4}{3} R_{3,1} - \frac{1}{3}R_{2,1}.
\end{equation}

Los elementos del arreglo $\mathbf{R}$ que hemos calculado hasta ahora son

\begin{equation}
\begin{bmatrix}
R_{1,1} & \\
R_{2,1} & 	R_{2,2} \\
R_{3,1} & 	R_{3,2}
\end{bmatrix}
\end{equation}

Ambos elementos de la segunda columna tienen un error de la forma $c_2h^4$, que también puede ser eliminado con extrapolación de Richardson.


Podemos usar $p=4$ en la ecuación para $G$ para tener

\begin{equation}
R_{3,3} = \frac{2^4 R_{3,2} - R_{2,2}}{2^{4-1}} = \frac{16}{15}R_{3,2} - \frac{1}{15}R_{2,2},
\end{equation}
que tiene un error de $\mathcal{O}(h^6)$. Con este, hemos extendido el arreglo a

\begin{equation}
\begin{bmatrix}
R_{1,1} &  & \\
R_{2,1} & 	R_{2,2} & \\
R_{3,1} & 	R_{3,2} & R_{3,3}
\end{bmatrix}
\end{equation}

Otra ronda de cálculos nos llevaría a un arreglo donde el error ya sería $\mathcal{O}(h^8)$

\begin{equation}
\begin{bmatrix}
R_{1,1} &  &  &\\
R_{2,1} & 	R_{2,2} &  &\\
R_{3,1} & 	R_{3,2} & R_{3,3} & \\
R_{4,1} & 	R_{4,2} & R_{4,3} &  R_{4,4}\\
\end{bmatrix}
\end{equation}

Noten que la aproximación más precisa de la integral es siempre el último término de la diagonal en este arreglo.  Este proceso se continua hasta que la diferencia entre dos términos sucesivos de la diagonal en el arreglo, sea lo suficientemente pequeña.

La fórmula general para la extrapolación en este esquema es

\begin{equation}
R_{i,j} = \frac{4^{j-1} R_{i,j-1} - R_{i-1,j-1}}{4^{j-1}-1}, ~~~i>1, ~~~ j=2,3,\ldots ,i
\end{equation}

Noten que despues de la primera extrapolación en la ecuación para $R_{2,2}$, ya no volvemos a usar $R_{1,1}$, entonces lo podemos reemplazar por $R_{2,2}$. Como resultado, tenemos que el arreglo ahora es de una dimensión

\begin{equation}
\begin{bmatrix}
R^\prime_{1} = R_{2,2} \\
R^\prime_{2} = R_{2,1} 	
\end{bmatrix} 
\end{equation}

En la segunda extrapolación reescribiríamos para tener

\begin{equation}
\begin{bmatrix}
R^\prime_{1} = R_{3,3} \\
R^\prime_{2} = R_{3,2} \\ 	
R^\prime_{3} = R_{3,1} 
\end{bmatrix} 
\end{equation}

Entonces para la $k$-ésima ronda, tenemos

\begin{equation}
R^\prime_{j} = \frac{4^{k-j} R^\prime_{j+1} - R^\prime_{j}}{4^{k-j}-1}, ~~~ j=k-1,k-2,\ldots ,1 \label{eq:romberg3}
\end{equation}



## Algoritmo de integración de Romberg

La función **romberg** nos regresa el resultado de la integral numérica y el número de paneles que se usaron. La extrapolación de Richardson se lleva a cabo con la subfunción **richardson**.

In [4]:
## modulo regla trapezoidal recursiva
'''Inew = trapecio_recursiva(f,a,b,Iold,k).
Iold = Integral de f(x) de x = a hasta b calculada
con la regla trapezoidal recursiva con 2ˆ(k-1) paneles.
Inew = la misma integral calculada con 2ˆk paneles.
'''
def trapecio_recursiva(f,a,b,Iold,k):
  if k == 1: Inew = (f(a) + f(b))*(b - a)/2.0
  else:
    n = 2**(k -2 ) # numero de nuevos puntos
    h = (b - a)/n # espaciamiento de nuevos puntos
    x = a + h/2.0
    sum = 0.0
    for i in range(n):
      sum = sum + f(x)
      x = x + h
      Inew = (Iold + h*sum)/2.0
  return Inew

## modulo romberg
'''I,nPanels = romberg(f,a,b,tol=1.0e-6).
Integracion de Romberg para f(x) en el rango x = a-b.
Regresa la integral y el numero de paneles usados.
Utiliza la regla trapezoidal recursiva.
'''

import numpy as np

def romberg(f,a,b,tol=1.0e-6):

  def richardson(r,k):
    for j in range(k-1,0,-1):
      const = 4.0**(k-j)
      r[j] = (const*r[j+1] - r[j])/(const - 1.0)
    return r
    
  r = np.zeros(21)
  r[1] = trapecio_recursiva(f,a,b,0.0,1)
  r_old = r[1]
  for k in range(2,21):
    r[k] = trapecio_recursiva(f,a,b,r[k-1],k)
    r = richardson(r,k)
    if abs(r[1]-r_old) < tol*max(abs(r[1]),1.0):
      return r[1],2**(k-1)
    r_old = r[1]
  print("La cuadratura de Romberg no converge")

# Ejemplo 1: Ejemplo con la Cuadratura de Romberg


Usa la regla trapezoidal recursiva para evaluar $\int_{0}^{\pi} \sqrt{x}
\cos x~ dx$ con hasta 6 decimales e indica cuantos paneles se necesitan para lograr este resultado. Aplicando el cambio de variable $t=\sqrt{x}$, ya teniamos una mejoria de más 



In [6]:
import math
#def g(x): return math.sqrt(x)*math.cos(x)
def g(x): return 2*x**2*math.cos(x**2)

I,n = romberg(g,0,math.sqrt(math.pi))
print('Integral =',I)
print('numEvals =',n)

Integral = -0.8948314695044126
numEvals = 64
